In [9]:
import os
import cv2
import dlib
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [10]:
# Define paths
training_path = '../dataset/train'
testing_path = '../dataset/test'
predictor_path = '../models/shape_predictor_68_face_landmarks.dat'

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)


In [11]:

def resize_image(image, target_size=(128, 128)):  # VGG uses 224x224 input size
    return cv2.resize(image, target_size, interpolation=cv2.INTER_LINEAR)


def align_face(image, landmarks):
    left_eye = landmarks[36]
    right_eye = landmarks[45]
    dx = right_eye[0] - left_eye[0]
    dy = right_eye[1] - left_eye[1]
    angle = np.degrees(np.arctan2(dy, dx))
    center = (int(np.mean([left_eye[0], right_eye[0]])),
              int(np.mean([left_eye[1], right_eye[1]])))

    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    return cv2.warpAffine(image, rotation_matrix, (image.shape[1], image.shape[0]))

In [12]:
# Extract facial landmarks from an image
def extract_landmarks(image_path):
    print(f"Processing image: {image_path}")

    image = cv2.imread(image_path)
    if image is None:
        print(f"Error loading image: {image_path}")
        return None

    image = resize_image(image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    detections = detector(gray)

    if len(detections) == 0:
        return None

    landmarks = predictor(gray, detections[0])
    landmarks = np.array([[p.x, p.y] for p in landmarks.parts()])
    image = align_face(image, landmarks)

    return image

def augment_image(image):


    return [image]


def extract_images_from_directory(directory):
    images = []
    labels = []
    for label in os.listdir(directory):
        label_path = os.path.join(directory, label)
        if not os.path.isdir(label_path):
            continue

        for img_file in os.listdir(label_path):
            img_path = os.path.join(label_path, img_file)
            image = extract_landmarks(img_path)
            if image is None:
                continue

            augmented_images = augment_image(image)
            for aug_image in augmented_images:
                images.append(aug_image)
                labels.append(label)

    return np.array(images), np.array(labels)



X_train, y_train = extract_images_from_directory(training_path)
X_test, y_test = extract_images_from_directory(testing_path)


X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0
# X_train = X_train.reshape(X_train.shape[0], 128, 128, 3)  # VGG input shape # Reshaping is done in extract_landmarks
# X_test = X_test.reshape(X_test.shape[0], 128, 128, 3)


label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)


y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))


for layer in vgg16_base.layers:
    layer.trainable = False



num_classes = y_train.shape[1]  #one-hot encoded
model = Sequential([
    vgg16_base,
    Flatten(),  # oe else use GlobalAveragePooling2D()
    Dense(512, activation='relu'),
    Dropout(0.48075087656540183),
    Dense(num_classes, activation='softmax')
])

Processing image: ../dataset/train\Heart\Heart(0).jpg
Processing image: ../dataset/train\Heart\Heart(1).jpg
Processing image: ../dataset/train\Heart\Heart(10).jpg
Processing image: ../dataset/train\Heart\Heart(100).jpg
Processing image: ../dataset/train\Heart\Heart(101).jpg
Processing image: ../dataset/train\Heart\Heart(102).jpg
Processing image: ../dataset/train\Heart\Heart(103).jpg
Processing image: ../dataset/train\Heart\Heart(104).jpg
Processing image: ../dataset/train\Heart\Heart(105).jpg
Processing image: ../dataset/train\Heart\Heart(106).jpg
Processing image: ../dataset/train\Heart\Heart(107).jpg
Processing image: ../dataset/train\Heart\Heart(108).jpg
Processing image: ../dataset/train\Heart\Heart(109).jpg
Processing image: ../dataset/train\Heart\Heart(11).jpg
Processing image: ../dataset/train\Heart\Heart(110).jpg
Processing image: ../dataset/train\Heart\Heart(111).jpg
Processing image: ../dataset/train\Heart\Heart(112).jpg
Processing image: ../dataset/train\Heart\Heart(113).jp

In [14]:
# Trial 2 finished with value: 0.7867305874824524 and parameters: {'learning_rate': 0.00013285427305222708, 'dense_units': 512, 'dropout_rate': 0.3004578707566693, 'batch_size': 32}. Best is trial 2 with value: 0.7867305874824524


model.compile(
    optimizer=Adam(learning_rate=0.00013285427305222708),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

checkpoint_callback = ModelCheckpoint(
    filepath='vgg16_model_{epoch:02d}.h5',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)


# early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test,y_test),
    epochs=90,
    batch_size=16,
    # callbacks=[early_stopping],
    verbose=1
)

test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Epoch 1/90
239/239 ━━━━━━━━━━━━━━━━━━━━ 539s 2s/step - accuracy: 0.3887 - loss: 1.4433 - val_accuracy: 0.5053 - val_loss: 1.2598
Epoch 2/90
239/239 ━━━━━━━━━━━━━━━━━━━━ 525s 2s/step - accuracy: 0.4907 - loss: 1.2319 - val_accuracy: 0.4727 - val_loss: 1.2572
Epoch 3/90
239/239 ━━━━━━━━━━━━━━━━━━━━ 457s 2s/step - accuracy: 0.5411 - loss: 1.1316 - val_accuracy: 0.5315 - val_loss: 1.1598
Epoch 4/90
239/239 ━━━━━━━━━━━━━━━━━━━━ 487s 2s/step - accuracy: 0.5896 - loss: 1.0364 - val_accuracy: 0.5588 - val_loss: 1.1015
Epoch 5/90
239/239 ━━━━━━━━━━━━━━━━━━━━ 529s 2s/step - accuracy: 0.6196 - loss: 0.9464 - val_accuracy: 0.5882 - val_loss: 1.0406
Epoch 6/90
239/239 ━━━━━━━━━━━━━━━━━━━━ 809s 3s/step - accuracy: 0.6797 - loss: 0.8608 - val_accuracy: 0.6019 - val_loss: 1.0248
Epoch 7/90
239/239 ━━━━━━━━━━━━━━━━━━━━ 812s 3s/step - accuracy: 0.6885 - loss: 0.8295 - val_accuracy: 0.6103 - val_loss: 0.9986
Epoch 8/90
239/239 ━━━━━━━━━━━━━━━━━━━━ 887s 4s/step - accuracy: 0.7206 - loss: 0.7663 - val_accu

In [15]:
# Save the model
model.save('../models/face_shape_classifier.h5')
print("Model saved to '../models/face_shape_classifier.h5'")

Model saved to '../models/face_shape_classifier.h5'


In [16]:
model.save('../models/face_shape_classifier.keras')
